In [1]:
import pandas as pd
import tensorflow as tf
import glob
import os
import numpy as np
import sklearn.metrics as sk
import tensorflow_datasets as tfds
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.losses import BinaryCrossentropy

import seaborn as sns
import helpers
import dataset
import matplotlib.pyplot as plt

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

2022-11-22 01:34:28.634214: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
lie_trial_path = './processed_lie/' #60 entries
truth_trial_path = './processed_truth/' #61 entries

# MU3D_path = '/Users/frank/Downloads/dataSets/MU3D-dataset/processed/' # 320 entries

In [3]:
X, Y = dataset.path_preprocessing(truth_trial_path, lie_trial_path)

In [4]:
TEST_RATIO = 0.2

xTrain, xTest = train_test_split(X, test_size=TEST_RATIO, shuffle=False)
yTrain, yTest = train_test_split(Y, test_size=TEST_RATIO, shuffle=False)

In [43]:
LSTM_NEURONS = 32
DROPOUT = 0
EPOCHS = 10
OPTIM = 'adam'
# LOSS = 'binary_crossentropy'
LOSS = BinaryCrossentropy(from_logits=True)

n_inputRows = xTrain.shape[0] 
n_timesteps = xTrain.shape[1]
n_features = xTrain.shape[2]

def LSTM_Model(neuro, drop, loss, optim, step, feat):
  model = Sequential()
  model.add(LSTM(neuro, return_sequences=False, batch_input_shape=(None, step, feat)))

  # model.add(Dropout(drop))
  model.add(Dense(1, activation='linear')) #softmax

  #model.add(Lambda(lambda x: tf.cast(tf.argmax(x), tf.float32)))
  #model.add(Dense(1, activation=activ))

  model.compile(loss=loss, optimizer=optim, metrics=['accuracy'])
  return model

MODEL = LSTM_Model(LSTM_NEURONS, DROPOUT, LOSS, OPTIM, n_timesteps, n_features)
MODEL.summary()
MODEL.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=EPOCHS, verbose=1)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 32)                6272      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 6,305
Trainable params: 6,305
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
1536/1536 [==============================] - 8s 4ms/step - loss: 0.0933 - accuracy: 0.9580 - val_loss: 0.0156 - val_accuracy: 0.9957
Epoch 2/10
1536/1536 [==============================] - 6s 4ms/step - loss: 0.0124 - accuracy: 0.9961 - val_loss: 0.0035 - val_accuracy: 0.9989
Epoch 3/10
1536/1536 [==============================] - 6s 4ms/step - loss: 0.0073 - accuracy: 0.9980 - val_loss: 0.0081 - val_accuracy: 0.9970
Epoch 4/10
1536/

In [64]:
xTrain

array([[[ 1.        , -0.13132551,  0.45949704, ...,  0.20938309,
          0.23263655,  1.        ],
        [ 1.        , -0.13587505,  0.46020457, ...,  0.2097182 ,
          0.23735671,  1.        ],
        [ 1.        , -0.14174735,  0.46926659, ...,  0.20999238,
          0.24207687,  1.        ],
        ...,
        [ 1.        , -0.15760056,  0.46894546, ...,  0.20938309,
          0.25691167,  1.        ],
        [ 1.        , -0.15998307,  0.47029524, ...,  0.20941356,
          0.25758597,  1.        ],
        [ 1.        , -0.1601632 ,  0.45513188, ...,  0.21035796,
          0.25354012,  1.        ]],

       [[ 1.        , -0.20496128, -0.00679299, ...,  0.13532369,
          0.23331086,  0.02777778],
        [ 1.        , -0.20143945, -0.01092348, ...,  0.13517136,
          0.22859069,  0.02777778],
        [ 1.        , -0.19598266, -0.00670124, ...,  0.13492765,
          0.22319622,  0.02777778],
        ...,
        [ 1.        , -0.14212528,  0.02791476, ...,  

In [66]:
import importlib
importlib.reload(test_lstm)

lie_test_path = './test/Lie/processed/'

test_lstm.perdictSingleVideo(lie_test_path, MODEL)

[array([[ 1.65284e-01,  3.22530e-02, -9.85718e-01,  8.60000e-02,
         1.60000e-01, -5.83000e+01,  5.58000e+01,  5.86600e+02,
         2.00000e-01,  5.00000e-02,  4.90000e-01,  2.40000e-01,
         2.60000e-01,  1.17000e+00,  7.18810e-02,  1.10446e-01,
         7.70000e-02],
       [ 1.68286e-01,  8.62900e-03, -9.85700e-01,  1.10000e-01,
         1.18000e-01, -5.90000e+01,  5.64000e+01,  5.85200e+02,
         1.95000e-01,  0.00000e+00,  5.10000e-01,  2.20000e-01,
         2.70000e-01,  9.20000e-01,  5.01710e-02,  1.15999e-01,
         1.08000e-01],
       [ 1.55668e-01,  3.19600e-03, -9.87804e-01,  1.08000e-01,
         9.30000e-02, -5.99000e+01,  5.73000e+01,  5.86300e+02,
         2.03000e-01,  0.00000e+00,  4.90000e-01,  3.10000e-01,
         2.90000e-01,  5.40000e-01,  5.22280e-02,  1.47934e-01,
         1.37000e-01],
       [ 1.35473e-01,  2.29420e-02, -9.90515e-01,  8.60000e-02,
         9.70000e-02, -6.08000e+01,  5.80000e+01,  5.86000e+02,
         2.06000e-01,  0.00000e+00

Traceback (most recent call last):
  File "/Users/frank/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3397, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/66/pn8653k51_75x5ksbpsy8hm80000gn/T/ipykernel_78230/2887895029.py", line 6, in <cell line: 6>
    test_lstm.perdictSingleVideo(lie_test_path, MODEL)
  File "/Users/frank/Downloads/Projects/Lie Detection/Lie_Detection/test_lstm.py", line 41, in perdictSingleVideo
    modelObj.predict(data)
  File "/Users/frank/opt/anaconda3/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/var/folders/66/pn8653k51_75x5ksbpsy8hm80000gn/T/__autograph_generated_file5e_jvs6i.py", line 15, in tf__predict_function
    retval_ = ag__.converted_call(ag__.ld(step_function), (ag__.ld(self), ag__.ld(iterator)), None, fscope)
ValueError: in user code:

    File "/Users/frank/opt/anaconda3/lib/p